In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from multiprocessing import Value
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from joblib import Parallel, delayed

In [7]:
df = pd.read_excel('../../../metrics.xlsx', nrows=100000)

In [8]:
print(len(df))

12945


In [9]:
df = df.drop(
    columns = [
        "Unnamed: 0", 
        "user", 
        'project', 
        'current_month', 
        'turnover_num',
        'turnover'
    ], axis = 1
)

In [10]:
max_float32 = np.finfo(np.float32).max
df[df > max_float32] = max_float32

In [11]:
df = df.replace([np.inf], np.nan).fillna(df.max())
df = df.replace([-np.inf], np.nan).fillna(df.min())

In [12]:
imputer = SimpleImputer(strategy='mean') 
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [13]:
x = df.drop(columns = ['time_to_stop_activity'], axis = 1).values
y = df['time_to_stop_activity'].values

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(
    x, y, 
    test_size = 0.3, 
    random_state=11
)

In [15]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
def train_evaluate_svr(regularization, tolerance, X_train, Y_train, X_test, Y_test):
    svr = SVR(kernel='rbf', C=regularization, epsilon=tolerance)
    svr.fit(X_train, Y_train)
    y_pred = svr.predict(X_test)
    mse = mean_squared_error(Y_test, y_pred)
    mae = mean_absolute_error(Y_test, y_pred)
    r2 = r2_score(Y_test, y_pred)
    
    return {
        'regularization': regularization, 
        'tolerance': tolerance, 
        'mean_squared_error': mse,
        'mean_absolute_error': mae,
        'r2_score': r2
    }

In [17]:
param_combinations = [
    (regularization, tolerance) 
    for regularization in [0.1, 1, 10, 100, 1000, 10000]
    for tolerance in [0.01, 0.05, 0.1, 0.2, 0.5, 1]
]

In [60]:
benchmark = Parallel(n_jobs=-1) (
    delayed(train_evaluate_svr) (reg, tol, X_train, Y_train, X_test, Y_test)
    for reg, tol in param_combinations
)

benchmark_df = pd.DataFrame(benchmark)

In [61]:
benchmark_df\
    .pivot(
        index='regularization', 
        columns='tolerance', 
        values='mean_squared_error'
    )

tolerance,0.01,0.05,0.10,0.20,0.50,1.00
regularization,,,,,,
0.1,17.287797,17.229818,17.081601,16.748890,15.828858,14.757449
1.0,13.299391,13.277055,13.215410,13.061348,12.545966,11.853835
10.0,11.422297,11.408934,11.365349,11.282742,10.934229,10.423376
100.0,10.605205,10.584964,10.534264,10.399913,9.978358,9.554725
1000.0,11.534280,11.503967,11.465096,11.320334,11.025576,10.754369
10000.0,23.076560,22.945781,22.933513,22.556385,21.746142,20.738345


In [62]:
benchmark_df\
    .pivot(
        index='regularization', 
        columns='tolerance', 
        values='mean_absolute_error'
    )

tolerance,0.01,0.05,0.10,0.20,0.50,1.00
regularization,,,,,,
0.1,2.430921,2.431836,2.433545,2.439294,2.472089,2.577342
1.0,2.209638,2.210906,2.213812,2.220804,2.254828,2.340061
10.0,2.040416,2.041425,2.043855,2.048689,2.087066,2.181845
100.0,1.937328,1.938033,1.939818,1.946753,1.981952,2.075233
1000.0,2.017774,2.019389,2.022046,2.027448,2.056038,2.132787
10000.0,2.554478,2.553456,2.557205,2.555304,2.567868,2.609438


In [63]:
benchmark_df\
    .pivot(
        index='regularization', 
        columns='tolerance', 
        values='r2_score'
    )

tolerance,0.01,0.05,0.10,0.20,0.50,1.00
regularization,,,,,,
0.1,-0.027473,-0.024027,-0.015218,0.004556,0.059237,0.122914
1.0,0.209571,0.210899,0.214563,0.223719,0.254350,0.295486
10.0,0.321134,0.321928,0.324518,0.329428,0.350141,0.380503
100.0,0.369696,0.370899,0.373913,0.381897,0.406952,0.432130
1000.0,0.314478,0.316280,0.318590,0.327194,0.344712,0.360831
10000.0,-0.371519,-0.363747,-0.363017,-0.340603,-0.292448,-0.232551


## Model Performance Improvement
Once the model has not a good performance, we are looking methods and techniques to improve the performance

##### **Kernel Trick**
SVM can model non-linear relationships between features by using the kernel trick. The default kernel is rbf (Radial Basis Function), but you can experiment with other kernels like:

- Linear Kernel: If your data is linearly separable, the linear kernel might be the best choice.
- Polynomial Kernel: Captures polynomial relationships between data points. You can control the degree of the polynomial to fit higher-order relationships.
- Sigmoid Kernel: Similar to a neural network activation function, this kernel maps data into a hyperbolic tangent space.

Action: Try different kernels and see which works best for your data. For non-linear data, rbf is usually a good default, but for linear data, a linear kernel might perform better.

In [46]:
def train_evaluate_svr(regularization, tolerance, kernel, X_train, Y_train, X_test, Y_test):
    
    svr = SVR(kernel=kernel, C=regularization, epsilon=tolerance)
    svr.fit(X_train, Y_train)
    y_pred = svr.predict(X_test)
    mse = mean_squared_error(Y_test, y_pred)
    mae = mean_absolute_error(Y_test, y_pred)
    r2 = r2_score(Y_test, y_pred)
    
    return {
        'kernel': kernel,
        'regularization': regularization, 
        'tolerance': tolerance, 
        'mean_squared_error': mse,
        'mean_absolute_error': mae,
        'r2_score': r2
    }

param_combinations = [
    (regularization, tolerance, kernel) 
    for regularization in [0.1, 1, 10, 100, 1000, 10000]
    for tolerance in [0.01, 0.05, 0.1, 0.2, 0.5, 1]
    for kernel in ['rbf', 'linear', 'poly']
]

total_tasks = len(param_combinations)

benchmark = Parallel(n_jobs=-1) (
    delayed(train_evaluate_svr) (reg, tol, kernel, X_train, Y_train, X_test, Y_test)
    for reg, tol, kernel in param_combinations
)

benchmark_df = pd.DataFrame(benchmark)

In [48]:
benchmark_df\
    [benchmark_df.kernel == 'rbf']\
    .pivot(
        index='regularization', 
        columns='tolerance', 
        values='mean_squared_error'
    )

tolerance,0.01,0.05,0.10,0.20,0.50,1.00
regularization,,,,,,
0.1,23.941913,23.7301,23.469835,22.964305,21.567714,19.640062
1.0,23.941913,23.7301,23.469835,22.964305,21.567714,19.640062
10.0,23.941913,23.7301,23.469835,22.964305,21.567714,19.640062
100.0,23.941913,23.7301,23.469835,22.964305,21.567714,19.640062
1000.0,23.941913,23.7301,23.469835,22.964305,21.567714,19.640062
10000.0,23.941913,23.7301,23.469835,22.964305,21.567714,19.640062


In [52]:
benchmark_df\
    [benchmark_df.kernel == 'linear']\
    .pivot(
        index='regularization', 
        columns='tolerance', 
        values='mean_squared_error'
    )

tolerance,0.01,0.05,0.10,0.20,0.50,1.00
regularization,,,,,,
0.1,3.076964e+168,3.076964e+168,3.076964e+168,3.076964e+168,3.076964e+168,3.076964e+168
1.0,3.076964e+170,3.076964e+170,3.076964e+170,3.076964e+170,3.076964e+170,3.076964e+170
10.0,3.076964e+172,3.076964e+172,3.076964e+172,3.076964e+172,3.076964e+172,3.076964e+172
100.0,3.076964e+174,3.076964e+174,3.076964e+174,3.076964e+174,3.076964e+174,3.076964e+174
1000.0,3.076964e+176,3.076964e+176,3.076964e+176,3.076964e+176,3.076964e+176,3.076964e+176
10000.0,3.076964e+178,3.076964e+178,3.076964e+178,3.076964e+178,3.076964e+178,3.076964e+178


In [50]:
benchmark_df\
    [benchmark_df.kernel == 'poly']\
    .pivot(
        index='regularization', 
        columns='tolerance', 
        values='mean_squared_error'
    )

tolerance,0.01,0.05,0.10,0.20,0.50,1.00
regularization,,,,,,
0.1,23.941913,23.7301,23.469835,22.964305,21.567714,19.640062
1.0,23.941913,23.7301,23.469835,22.964305,21.567714,19.640062
10.0,23.941913,23.7301,23.469835,22.964305,21.567714,19.640062
100.0,23.941913,23.7301,23.469835,22.964305,21.567714,19.640062
1000.0,23.941913,23.7301,23.469835,22.964305,21.567714,19.640062
10000.0,23.941913,23.7301,23.469835,22.964305,21.567714,19.640062


##### **Feature Scaling**
SVM is sensitive to the scale of the features. Features with larger numerical ranges dominate the decision boundary, so proper scaling is essential. You are already using StandardScaler, but you can also try:

- MinMaxScaler: Rescales features into a range, typically [0, 1]. This might work better if your features have different scales.
- RobustScaler: This is more robust to outliers because it scales the data based on the median and the interquartile range instead of the mean and standard deviation.

Action: Experiment with different scalers and check how each affects the performance.

In [26]:
X_train, X_test, Y_train, Y_test = train_test_split(
    x, y, 
    test_size = 0.3, 
    random_state = 11
)

In [27]:
def train_evaluate_svr(regularization, tolerance, scaler, X_train, Y_train, X_test, Y_test):
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    svr = SVR(kernel='rbf', C=regularization, epsilon=tolerance)
    svr.fit(X_train, Y_train)

    y_pred = svr.predict(X_test)
    mse = mean_squared_error(Y_test, y_pred)
    mae = mean_absolute_error(Y_test, y_pred)
    r2 = r2_score(Y_test, y_pred)
    
    return {
        'scaler': type(scaler).__name__,
        'regularization': regularization, 
        'tolerance': tolerance, 
        'mean_squared_error': mse,
        'mean_absolute_error': mae,
        'r2_score': r2
    }

In [28]:
param_combinations = [
    (regularization, tolerance, scaler) 
    for regularization in [0.1, 1, 10, 100, 1000, 10000]
    for tolerance in [0.01, 0.05, 0.1, 0.2, 0.5, 1]
    for scaler in [StandardScaler(), MinMaxScaler(), RobustScaler()]
]

In [29]:
benchmark = Parallel(n_jobs=-1) (
    delayed(train_evaluate_svr) (reg, tol, scaler, X_train, Y_train, X_test, Y_test)
    for reg, tol, scaler in param_combinations
)

benchmark_df = pd.DataFrame(benchmark)

In [41]:
benchmark_df\
    [benchmark_df.scaler == 'StandardScaler']\
    .pivot(
        index='regularization', 
        columns='tolerance', 
        values='mean_squared_error'
    )

tolerance,0.01,0.05,0.10,0.20,0.50,1.00
regularization,,,,,,
0.1,17.287797,17.229818,17.081601,16.748890,15.828858,14.757449
1.0,13.299391,13.277055,13.215410,13.061348,12.545966,11.853835
10.0,11.422297,11.408934,11.365349,11.282742,10.934229,10.423376
100.0,10.605205,10.584964,10.534264,10.399913,9.978358,9.554725
1000.0,11.534280,11.503967,11.465096,11.320334,11.025576,10.754369
10000.0,23.076560,22.945781,22.933513,22.556385,21.746142,20.738345


In [45]:
benchmark_df\
    [benchmark_df.scaler == 'RobustScaler']\
    .pivot(
        index='regularization', 
        columns='tolerance', 
        values='mean_squared_error'
    )

tolerance,0.01,0.05,0.10,0.20,0.50,1.00
regularization,,,,,,
0.1,23.941913,23.7301,23.469835,22.964305,21.567714,19.640062
1.0,23.941913,23.7301,23.469835,22.964305,21.567714,19.640062
10.0,23.941913,23.7301,23.469835,22.964305,21.567714,19.640062
100.0,23.941913,23.7301,23.469835,22.964305,21.567714,19.640062
1000.0,23.941913,23.7301,23.469835,22.964305,21.567714,19.640062
10000.0,23.941913,23.7301,23.469835,22.964305,21.567713,19.640061


In [44]:
benchmark_df\
    [benchmark_df.scaler == 'MinMaxScaler']\
    .pivot(
        index='regularization', 
        columns='tolerance', 
        values='mean_squared_error'
    )

tolerance,0.01,0.05,0.10,0.20,0.50,1.00
regularization,,,,,,
0.1,21.400989,21.338895,21.122145,20.737728,19.447737,18.384828
1.0,18.307879,18.204993,18.043665,17.644855,16.546957,15.276378
10.0,15.023713,14.981504,14.890117,14.678560,14.022465,13.148012
100.0,13.054013,13.038923,13.013252,12.883137,12.379654,11.777031
1000.0,11.923947,11.939167,11.909038,11.810606,11.467938,11.034528
10000.0,12.381233,12.365877,12.316832,12.224160,11.801225,11.451055


#### Handling Outliers
SVM is highly sensitive to outliers, as they can significantly influence the decision boundary and margins. Outliers can distort the hyperplane, leading to poor generalization.

- Outlier Detection: Before training the model, perform outlier detection (e.g., using Z-scores, IQR, or visualizations like box plots) and remove or adjust them.
- Adjust Epsilon: If your model has many outliers, you may want to increase epsilon to create a wider margin and minimize the effect of outliers.

Action: Identify and handle outliers by removing or transforming them to see if performance improves.